In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unique-videos/unique_videos.csv
/kaggle/input/duration-5min-1sthalf/duration_5min_1stHalf.xlsx


In [2]:
pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-7zehndo7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-7zehndo7
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802826 sha256=772820e027e539c76a07f0fb77d257f0406c2f6e19ab962d3452ed0c5569861d
  Stored in directory: /tmp/pip-ephem-wheel-cache-fsgvpu8m/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Note: you may need to restart the kernel to use

In [3]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.7 MB/s eta 0:00:00


In [4]:
import warnings
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU; using FP32 instead")

import os
import tempfile
import subprocess
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import whisper
from tqdm import tqdm

def download_audio(video_id, retries=3):
    """Download the audio from a YouTube video using yt-dlp and convert it to WAV format."""
    output_path_wav = f"{tempfile.mktemp(suffix='.wav')}"
    command = [
        'yt-dlp',
        '-x', '--audio-format', 'wav',
        '--audio-quality', '9',  
        '-o', output_path_wav,
        f'https://www.youtube.com/watch?v={video_id}'
    ]
    for attempt in range(retries):
        try:
            result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            return output_path_wav
        except subprocess.CalledProcessError as e:
            print(f"Attempt {attempt + 1} failed for video ID {video_id}: {e.stderr.decode().strip()}")
            if attempt == retries - 1:
                print(f"Failed to download or convert audio for video ID {video_id} after {retries} attempts.")
                print(f"Error message: {e.stderr.decode().strip()}")
    return None

def transcribe_audio(audio_path):
    """Transcribe audio using Whisper."""
    try:
        model = whisper.load_model("tiny")
        result = model.transcribe(audio_path)
        return result['text']
    except Exception as e:
        print(f"Failed to transcribe audio: {e}")
        return "Transcription failed"

def process_video(video_id, channel_id):
    """Process a single video given by video_id and include channel_id."""
    audio_path = download_audio(video_id)
    if audio_path:
        transcription = transcribe_audio(audio_path)
        os.remove(audio_path)
    else:
        transcription = "No transcription available"
    return video_id, channel_id, transcription

def process_videos(video_data):
    """Download and transcribe videos listed by their YouTube IDs using multithreading, including channel IDs."""
    transcriptions = []
    output_csv = "/kaggle/working/test.csv"
    
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_video = {executor.submit(process_video, row['VideoID'], row['channelId']): row for index, row in video_data.iterrows()}
        for i, future in tqdm(enumerate(as_completed(future_to_video), start=1), total=len(video_data), desc="Processing videos"):
            try:
                video_id, channel_id, transcription = future.result()
            except Exception as e:
                print(f"Error processing video: {e}")
                video_id = future_to_video[future]['VideoID']
                channel_id = future_to_video[future]['channelId']
                transcription = "Error during processing"
            transcriptions.append({'VideoID': video_id, 'ChannelID': channel_id, 'Transcription': transcription})
            # Save transcriptions to CSV file after each iteration
            pd.DataFrame(transcriptions).to_csv(output_csv, index=False)
    
    return pd.DataFrame(transcriptions)
# Example usage
df2 = pd.read_csv('/kaggle/input/unique-videos/unique_videos.csv')
# df2 = df2[24285:25000]
df = process_videos(df2)
df.to_csv("/kaggle/working/duration_5min_2ndstHalf-unique.csv", index=False)

Processing videos:   0%|          | 0/4246 [00:00<?, ?it/s]
  0%|                                              | 0.00/72.1M [00:00<?, ?iB/s]/opt/conda/lib/python3.10/site-packages/whisper/__init__.py:65: UserWarning: /root/.cache/whisper/tiny.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(

  7%|██▍                                  | 4.81M/72.1M [00:00<00:01, 50.1MiB/s]

  0%|                                              | 0.00/72.1M [00:00<?, ?iB/s]
 31%|███████████▋                          | 22.1M/72.1M [00:00<00:00, 127MiB/s]

  8%|██▉                                  | 5.78M/72.1M [00:00<00:01, 60.6MiB/s]
 53%|████████████████████▎                 | 38.5M/72.1M [00:00<00:00, 147MiB/s]

 24%|████████▋                            | 17.0M/72.1M [00:00<00:00, 94.1MiB/s]
 73%|███████████████████████████▋          | 52.5M/72.1M [00:00<00:00, 147MiB/s]

 36%|█████████████▍                       | 26.2M/72.1M [00:00<00:00, 95.2MiB/s]
100%|███████

Attempt 1 failed for video ID Rc4jWMznRqQ: ERROR: [youtube] Rc4jWMznRqQ: Private video. Sign in if you've been granted access to this video
Attempt 2 failed for video ID Rc4jWMznRqQ: ERROR: [youtube] Rc4jWMznRqQ: Private video. Sign in if you've been granted access to this video


Processing videos:   5%|▌         | 233/4246 [30:45<8:44:34,  7.84s/it] 

Attempt 3 failed for video ID Rc4jWMznRqQ: ERROR: [youtube] Rc4jWMznRqQ: Private video. Sign in if you've been granted access to this video
Failed to download or convert audio for video ID Rc4jWMznRqQ after 3 attempts.
Error message: ERROR: [youtube] Rc4jWMznRqQ: Private video. Sign in if you've been granted access to this video


Processing videos:   7%|▋         | 276/4246 [34:41<3:51:16,  3.50s/it]

...

Processing videos:  89%|████████▉ | 3797/4246 [7:46:28<43:23,  5.80s/it]

Attempt 1 failed for video ID i2tlB19ru7c: ERROR: unable to download video data: HTTP Error 403: Forbidden


Processing videos:  93%|█████████▎| 3944/4246 [8:04:29<33:54,  6.74s/it]

Attempt 1 failed for video ID Y46ZmxUhOzI: ERROR: unable to download video data: HTTP Error 403: Forbidden


Processing videos: 100%|██████████| 4246/4246 [8:43:36<00:00,  7.40s/it]


In [5]:
# import warnings
# warnings.filterwarnings("ignore", message="FP16 is not supported on CPU; using FP32 instead")

# import os
# import tempfile
# import subprocess
# import pandas as pd
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import whisper
# from tqdm import tqdm

# def download_audio(video_id):
#     """Download the audio from a YouTube video using yt-dlp and convert it to WAV format."""
#     output_path_wav = f"{tempfile.mktemp(suffix='.wav')}"
#     try:
#         command = [
#             'yt-dlp',
#             '-x', '--audio-format', 'wav',
#             '--audio-quality', '0',  # Best quality
#             '-o', output_path_wav,
#             f'https://www.youtube.com/watch?v={video_id}'
#         ]
#         subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#         return output_path_wav
#     except subprocess.CalledProcessError as e:
#         print(f"Failed to download or convert audio for video ID {video_id}: {e}")
#         return None

# def transcribe_audio(audio_path):
#     """Transcribe audio using Whisper."""
#     try:
#         model = whisper.load_model("tiny")
#         result = model.transcribe(audio_path)
#         return result['text']
#     except Exception as e:
#         print(f"Failed to transcribe audio: {e}")
#         return "Transcription failed"

# def process_video(video_id, channel_id):
#     """Process a single video given by video_id and include channel_id."""
#     audio_path = download_audio(video_id)
#     if audio_path:
#         transcription = transcribe_audio(audio_path)
#         os.remove(audio_path)
#     else:
#         transcription = "No transcription available"
#     return video_id, channel_id, transcription

# def process_videos(video_data):
#     """Download and transcribe videos listed by their YouTube IDs using multithreading, including channel IDs."""
#     transcriptions = []
#     output_csv = "/kaggle/working/test.csv"
    
#     with ThreadPoolExecutor(max_workers=3) as executor:
#         future_to_video = {executor.submit(process_video, row['id'], row['channelId']): row for index, row in video_data.iterrows()}
#         for i, future in tqdm(enumerate(as_completed(future_to_video), start=1), total=len(video_data), desc="Processing videos"):
#             try:
#                 video_id, channel_id, transcription = future.result()
#             except Exception as e:
#                 print(f"Error processing video: {e}")
#                 video_id = future_to_video[future]['id']
#                 channel_id = future_to_video[future]['channelId']
#                 transcription = "Error during processing"
#             transcriptions.append({'VideoID': video_id, 'ChannelID': channel_id, 'Transcription': transcription})
#             # Save transcriptions to CSV file after each iteration
#             pd.DataFrame(transcriptions).to_csv(output_csv, index=False)
    
#     return pd.DataFrame(transcriptions)

# # Example usage
# df2 = pd.read_excel('/kaggle/input/duration-5min-1sthalf/duration_5min_1stHalf.xlsx')
# df_slice = df2.iloc[10000:15000]  # Selects rows 5001 to 7000
#   # Load only the first 50 for example, assuming 'id' and 'channelId' are columns
# df = process_videos(df_slice)
# df.to_csv("/kaggle/working/duration_5min_2ndstHalf10000-15000.csv", index=False)